In [62]:
using CUDA; 
using Knet
using MLDatasets
include("resnet_final.jl")

ResNet

In [64]:
function loss_nll(model, x, y)
    scores = model(x)
    loss = nll(scores, y)
    loss
end

loss_nll (generic function with 1 method)

In [65]:
function validate(model, dtst)
    acc = 0.0
    batch_count = 0
    for (x, y) in dtst
        scores = model(x)
        acc += accuracy(scores, y)
        batch_count += 1
    end
    acc /= batch_count
end


validate (generic function with 1 method)

In [66]:
function load_cifar_dataset()
    xtrn,ytrn = CIFAR10.traindata(Float32);
    ytrn = ytrn .+ 1
    xtrn = permutedims(xtrn, (2, 1, 3, 4));
    xtst,ytst = CIFAR10.testdata(Float32);
    ytst = ytst .+ 1
    xtst = permutedims(xtst, (2, 1, 3, 4));
    xtrn, ytrn, xtst, ytst
end

load_cifar_dataset (generic function with 1 method)

In [67]:
bsize = 256
xtrn, ytrn, xtst, ytst = load_cifar_dataset();

In [68]:
model = ResNet(depth=18, in_ch=3, out_ch=10, cifar_stem=true)

Sequential(Any[Conv2d(P(KnetArray{Float32,4}(3,3,3,64)), nothing, 1, 1, identity), BatchNorm2d(Knet.Ops20.BNMoments(0.1, nothing, nothing, zeros, ones), K32(128)[1.0⋯], Knet.Ops20.relu), Sequential(Any[ResidualBlock(Sequential(Any[Conv2d(P(KnetArray{Float32,4}(3,3,64,64)), nothing, 1, 1, identity), BatchNorm2d(Knet.Ops20.BNMoments(0.1, nothing, nothing, zeros, ones), K32(128)[1.0⋯], Knet.Ops20.relu), Conv2d(P(KnetArray{Float32,4}(3,3,64,64)), nothing, 1, 1, identity), BatchNorm2d(Knet.Ops20.BNMoments(0.1, nothing, nothing, zeros, ones), K32(128)[1.0⋯], identity)]), identity), ResidualBlock(Sequential(Any[Conv2d(P(KnetArray{Float32,4}(3,3,64,64)), nothing, 1, 1, identity), BatchNorm2d(Knet.Ops20.BNMoments(0.1, nothing, nothing, zeros, ones), K32(128)[1.0⋯], Knet.Ops20.relu), Conv2d(P(KnetArray{Float32,4}(3,3,64,64)), nothing, 1, 1, identity), BatchNorm2d(Knet.Ops20.BNMoments(0.1, nothing, nothing, zeros, ones), K32(128)[1.0⋯], identity)]), identity)]), Sequential(Any[ResidualBlock(Seque

In [82]:
for p in params(model)
    p.opt = Adam(lr=0.000001)
end

In [80]:
epochs = 20
train_loss = zeros(epochs);
val_acc = zeros(epochs);

In [81]:
for i in 1:epochs
    print("\nEpoch: ", i)
    # This is done without minimize function for easier debugging
    train_loss[i] = 0.0
    batch_count = 0
    dtrn = minibatch(xtrn, ytrn, bsize,shuffle=true)
    for (x, y) in dtrn
        loss = @diff loss_nll(model, x, y)
        train_loss[i] += value(loss)
        batch_count += 1
        for p in params(model)
            g = grad(loss, p)
            update!(value(p), g, p.opt)
        end
    end
    train_loss[i] /= batch_count
    print(",Train epoch loss: ", train_loss[i])
    if mod(i, 5) == 0
        val_batch_count = 0
        dtst = minibatch(xtst, ytst, bsize)
        val_acc[i] = validate(model, dtst)
        print(",Accuracy: ", val_acc[i])
    end
    #GC.gc()
end


Epoch: 1,Train epoch loss: 5.36269866503202e-7
Epoch: 2,Train epoch loss: 5.358686813941368e-7
Epoch: 3,Train epoch loss: 5.31627581669734e-7
Epoch: 4,Train epoch loss: 5.049010117848714e-7
Epoch: 5,Train epoch loss: 5.452296672723232e-7,Accuracy: 0.8631810897435898
Epoch: 6,Train epoch loss: 5.534444099817521e-7
Epoch: 7,Train epoch loss: 5.608185743674253e-7
Epoch: 8,Train epoch loss: 5.066776886964455e-7
Epoch: 9,Train epoch loss: 5.03907601038615e-7
Epoch: 10,Train epoch loss: 4.963806042304406e-7,Accuracy: 0.862479967948718
Epoch: 11,Train epoch loss: 5.086263020833333e-7
Epoch: 12,Train epoch loss: 4.961704596495017e-7
Epoch: 13,Train epoch loss: 5.362889705560146e-7
Epoch: 14,Train epoch loss: 4.724623301090338e-7
Epoch: 15,Train epoch loss: 4.89885226274148e-7,Accuracy: 0.8638822115384616
Epoch: 16,Train epoch loss: 5.264885914631379e-7
Epoch: 17,Train epoch loss: 4.507792301667042e-7
Epoch: 18,Train epoch loss: 4.6424758739960497e-7
Epoch: 19,Train epoch loss: 5.0354462403517

In [78]:
for i in 1:epochs
    print("\nEpoch: ", i)
    # This is done without minimize function for easier debugging
    train_loss[i] = 0.0
    batch_count = 0
    dtrn = minibatch(xtrn, ytrn, bsize,shuffle=true)
    for (x, y) in dtrn
        loss = @diff loss_nll(model, x, y)
        train_loss[i] += value(loss)
        batch_count += 1
        for p in params(model)
            g = grad(loss, p)
            update!(value(p), g, p.opt)
        end
    end
    train_loss[i] /= batch_count
    print(",Train epoch loss: ", train_loss[i])
    if mod(i, 5) == 0
        val_batch_count = 0
        dtst = minibatch(xtst, ytst, bsize)
        val_acc[i] = validate(model, dtst)
        print(",Accuracy: ", val_acc[i])
    end
    #GC.gc()
end


Epoch: 1,Train epoch loss: 2.438078324000041e-6
Epoch: 2,Train epoch loss: 2.092314072144337e-6
Epoch: 3,Train epoch loss: 2.0559208515362862e-6
Epoch: 4,Train epoch loss: 1.9563123201712582e-6
Epoch: 5,Train epoch loss: 1.564927590199006e-6,Accuracy: 0.8623798076923077
Epoch: 6,Train epoch loss: 1.4326702325772016e-6
Epoch: 7,Train epoch loss: 1.4886451073181935e-6
Epoch: 8,Train epoch loss: 1.2886638824756328e-6
Epoch: 9,Train epoch loss: 1.1931818265181322e-6
Epoch: 10,Train epoch loss: 1.0497677020537547e-6,Accuracy: 0.8625801282051282
Epoch: 11,Train epoch loss: 1.1498729387919109e-6
Epoch: 12,Train epoch loss: 1.1563492126953908e-6
Epoch: 13,Train epoch loss: 9.284760707463974e-7
Epoch: 14,Train epoch loss: 8.302239271310659e-7
Epoch: 15,Train epoch loss: 7.54323525306506e-7,Accuracy: 0.8626802884615384
Epoch: 16,Train epoch loss: 8.008036857996232e-7
Epoch: 17,Train epoch loss: 7.254955096122546e-7
Epoch: 18,Train epoch loss: 7.019975246527256e-7
Epoch: 19,Train epoch loss: 6.4

In [75]:
for i in 1:epochs
    print("\nEpoch: ", i)
    # This is done without minimize function for easier debugging
    train_loss[i] = 0.0
    batch_count = 0
    dtrn = minibatch(xtrn, ytrn, bsize,shuffle=true)
    for (x, y) in dtrn
        loss = @diff loss_nll(model, x, y)
        train_loss[i] += value(loss)
        batch_count += 1
        for p in params(model)
            g = grad(loss, p)
            update!(value(p), g, p.opt)
        end
    end
    train_loss[i] /= batch_count
    print(",Train epoch loss: ", train_loss[i])
    if mod(i, 5) == 0
        val_batch_count = 0
        dtst = minibatch(xtst, ytst, bsize)
        val_acc[i] = validate(model, dtst)
        print(",Accuracy: ", val_acc[i])
    end
    #GC.gc()
end


Epoch: 1,Train epoch loss: 1.737085672525259e-5
Epoch: 2,Train epoch loss: 1.6480340407444882e-5
Epoch: 3,Train epoch loss: 1.4341679903177114e-5
Epoch: 4,Train epoch loss: 1.3040464658003587e-5
Epoch: 5,Train epoch loss: 1.1288699431297106e-5,Accuracy: 0.8613782051282052
Epoch: 6,Train epoch loss: 1.1735562330637223e-5
Epoch: 7,Train epoch loss: 9.927497460291937e-6
Epoch: 8,Train epoch loss: 8.786507906057897e-6
Epoch: 9,Train epoch loss: 8.203261173688448e-6
Epoch: 10,Train epoch loss: 6.793840573384211e-6,Accuracy: 0.8616786858974359
Epoch: 11,Train epoch loss: 6.289321642655593e-6
Epoch: 12,Train epoch loss: 5.8398605921329596e-6
Epoch: 13,Train epoch loss: 5.252028887088482e-6
Epoch: 14,Train epoch loss: 4.77295655470628e-6
Epoch: 15,Train epoch loss: 4.180558981039585e-6,Accuracy: 0.8621794871794872
Epoch: 16,Train epoch loss: 4.280530489408053e-6
Epoch: 17,Train epoch loss: 3.6356922907707017e-6
Epoch: 18,Train epoch loss: 3.085801234612098e-6
Epoch: 19,Train epoch loss: 2.791

In [74]:
for i in 1:epochs
    print("\nEpoch: ", i)
    # This is done without minimize function for easier debugging
    train_loss[i] = 0.0
    batch_count = 0
    dtrn = minibatch(xtrn, ytrn, bsize,shuffle=true)
    for (x, y) in dtrn
        loss = @diff loss_nll(model, x, y)
        train_loss[i] += value(loss)
        batch_count += 1
        for p in params(model)
            g = grad(loss, p)
            update!(value(p), g, p.opt)
        end
    end
    train_loss[i] /= batch_count
    print(",Train epoch loss: ", train_loss[i])
    if mod(i, 5) == 0
        val_batch_count = 0
        dtst = minibatch(xtst, ytst, bsize)
        val_acc[i] = validate(model, dtst)
        print(",Accuracy: ", val_acc[i])
    end
    #GC.gc()
end


Epoch: 1,Train epoch loss: 0.006886395296225181
Epoch: 2,Train epoch loss: 0.0009877040218084286
Epoch: 3,Train epoch loss: 0.00042290513714154563
Epoch: 4,Train epoch loss: 0.00030093461466141236
Epoch: 5,Train epoch loss: 0.000208189539038218,Accuracy: 0.8595753205128205
Epoch: 6,Train epoch loss: 0.00015340069165596594
Epoch: 7,Train epoch loss: 0.00012459147434968216
Epoch: 8,Train epoch loss: 0.00010527610205687009
Epoch: 9,Train epoch loss: 8.85431774151631e-5
Epoch: 10,Train epoch loss: 6.992263862719903e-5,Accuracy: 0.859375
Epoch: 11,Train epoch loss: 6.169994863180014e-5
Epoch: 12,Train epoch loss: 5.514879639332111e-5
Epoch: 13,Train epoch loss: 4.525164572092203e-5
Epoch: 14,Train epoch loss: 3.674099078545204e-5
Epoch: 15,Train epoch loss: 3.593911727269491e-5,Accuracy: 0.8620793269230769
Epoch: 16,Train epoch loss: 3.0409048000971477e-5
Epoch: 17,Train epoch loss: 2.7837871741025877e-5
Epoch: 18,Train epoch loss: 2.3794785524025942e-5
Epoch: 19,Train epoch loss: 2.170466

In [71]:
for i in 1:epochs
    print("\nEpoch: ", i)
    # This is done without minimize function for easier debugging
    train_loss[i] = 0.0
    batch_count = 0
    dtrn = minibatch(xtrn, ytrn, bsize,shuffle=true)
    for (x, y) in dtrn
        loss = @diff loss_nll(model, x, y)
        train_loss[i] += value(loss)
        batch_count += 1
        for p in params(model)
            g = grad(loss, p)
            update!(value(p), g, p.opt)
        end
    end
    train_loss[i] /= batch_count
    print(",Train epoch loss: ", train_loss[i])
    if mod(i, 5) == 0
        val_batch_count = 0
        dtst = minibatch(xtst, ytst, bsize)
        val_acc[i] = validate(model, dtst)
        print(",Accuracy: ", val_acc[i])
    end
    #GC.gc()
end


Epoch: 1,Train epoch loss: 1.3433236635648287
Epoch: 2,Train epoch loss: 0.7899385363627702
Epoch: 3,Train epoch loss: 0.5810251770875393
Epoch: 4,Train epoch loss: 0.44252078594305577
Epoch: 5,Train epoch loss: 0.342194070877173,Accuracy: 0.7380809294871795
Epoch: 6,Train epoch loss: 0.2610907916839306
Epoch: 7,Train epoch loss: 0.18539956338130512
Epoch: 8,Train epoch loss: 0.14339740971724194
Epoch: 9,Train epoch loss: 0.11612590740506466
Epoch: 10,Train epoch loss: 0.08744526887551332,Accuracy: 0.8005809294871795
Epoch: 11,Train epoch loss: 0.075722463371662
Epoch: 12,Train epoch loss: 0.06354978800966189
Epoch: 13,Train epoch loss: 0.058929918009119155
Epoch: 14,Train epoch loss: 0.0543970759671468
Epoch: 15,Train epoch loss: 0.04480961330999166,Accuracy: 0.8139022435897436
Epoch: 16,Train epoch loss: 0.039163977662340185
Epoch: 17,Train epoch loss: 0.04437785453330248
Epoch: 18,Train epoch loss: 0.04794140994930879
Epoch: 19,Train epoch loss: 0.03559470335260416
Epoch: 20,Train 